In [2]:
from __future__ import print_function
from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.layers import LSTM, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib.dates import DateFormatter
import pandas
import keras
import numpy as np
import matplotlib.pyplot as plt
import mock
import datetime
import matplotlib.dates as mdates
import tensorflow as tf

In [3]:
def read_dataset(filename, columns, scaler):
    dataframe = pandas.read_csv(filename, usecols=columns, engine='python')
    dataset = dataframe.values
    #dataset = scaler.fit_transform(dataset)
    return dataset

In [4]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i+time_step,0])
    return np.array(dataX), np.array(dataY)

In [5]:
def save_model(filename, model):
    fn_json = filename + '.json'
    fn_h5 = filename + '.h5'
    model_json = model.to_json()
    with open(fn_json, 'w') as json_file:
        json_file.write(model_json)
    model.save_weights(fn_h5)

In [6]:
def hard_tanh (x):
    return tf.minimum(tf.maximum(x, -1.), 1.)

In [7]:
old_get = keras.activations.get
def patch_get(x):
    return hard_tanh if x == 'hard_tanh' else old_get(x)
@mock.patch('keras.activations.get', patch_get)
def load_model(filename):
    fn_json = filename + '.json'
    fn_h5 = filename + '.h5'
    json_file = open(fn_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json, custom_objects={'hard_tanh': hard_tanh})
    loaded_model.load_weights(fn_h5)
    print('Model is loaded')
    return loaded_model

In [8]:
def noisy_input(x, c=0.05):
    noise = np.random.normal(loc=0.0, scale=1.0, size=np.shape(x))
    output = x + c*noise
    return output

In [9]:
def model_lstm_train(units, trainX, trainY, epochs, batch_size, validation_split):
    model = Sequential()
    model.add(LSTM(units, input_shape=(time_step, 1)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=2, validation_split=validation_split)

    save_model('models/LSTM_model', model)

    return model

In [10]:
def model_gru_train(units, trainX, trainY, epochs, batch_size, validation_split):
    model = Sequential()
    model.add(GRU(units, input_shape=(time_step, 1)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=2, validation_split=validation_split)

    save_model('models/GRU_model',model)

    return model


In [11]:
def flow_predict(x_data, x_true, model, model_type):
    dataPredict = model.predict(x_data)

    #dataPredict = scaler.inverse_transform(dataPredict)
    #x_true = scaler.inverse_transform([x_true])

    scoreMSE = mean_squared_error(x_true, dataPredict[:,0])
    scoreMAE = mean_absolute_error(x_true, dataPredict[:,0])

    print(model_type + 'Test score: %.2lf MSE %.2lf MAE' %(scoreMSE, scoreMAE))

    return dataPredict, x_true, scoreMSE, scoreMAE

In [12]:
def flow_plot(x_pred, x_true, start, len):
    truePlot = np.transpose(x_true)
    truePlot = truePlot[start:len]
    predPlot = x_pred[start:len]

    plt.plot(truePlot, label='True Plot')
    plt.plot(predPlot, label='Predicted Plot')
    plt.legend()
    plt.show()

#Set so results don't change every time it is run
np.random.seed(7)

In [13]:
scaler = MinMaxScaler(feature_range=(-1, 1))

In [15]:
train = read_dataset('https://raw.githubusercontent.com/fjbriones/coe197-project/master/data/train_dataset.csv', [1], scaler)
test = read_dataset('https://raw.githubusercontent.com/fjbriones/coe197-project/master/data/test_dataset.csv', [1], scaler)

In [16]:
time_step = 6
len_day = 288 #len_day in terms of 5 minutes
# epochs = 500
batch_size = 32
units = 32
validation_split = 0.2

#Create datasets
trainX, trainY = create_dataset(train, time_step)
testX, testY = create_dataset(test, time_step)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

#For optimal training, search for the right number of iterations.
start = 20
end = 320
step = 20

n = (int)((end - start)/step)

data_train_mse = np.zeros((6, n))
data_test_mse = np.zeros((6, n))
data_train_mae = np.zeros((6, n))
data_test_mae = np.zeros((6, n))


IndexError: index 0 is out of bounds for axis 1 with size 0